In [ ]:
!pip install wrds

     |████████████████████████████████| 3.0 MB 33.7 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import matplotlib.pyplot as plt
import scipy.stats as stats
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

In [ ]:
# connect to wrds
conn = wrds.Connection()

Enter your WRDS username [root]:poonehmg
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
Loading library list...
Done


In [ ]:
#Downloading CRPS Data - Monthly File
#The varialbes are permenant company number, date, share code, exchange code, return, return exluding dividends,
#shares outstanding and price for NYSE, NASDAQ and AMEX stocks and Merge with historical header using crsp.msenames
crsp = conn.raw_sql("""
                      select *
                      from crsp.dsf as a
                      left join crsp.dsenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '12/30/2021' and '12/31/2021'
                      and b.exchcd between 1 and 3
                      """, date_cols=['date'])

In [ ]:
crsp.columns

Index(['cusip', 'permno', 'permco', 'issuno', 'hexcd', 'hsiccd', 'date',
       'bidlo', 'askhi', 'prc', 'vol', 'ret', 'bid', 'ask', 'shrout', 'cfacpr',
       'cfacshr', 'openprc', 'numtrd', 'retx', 'permno', 'namedt', 'nameendt',
       'shrcd', 'exchcd', 'siccd', 'ncusip', 'ticker', 'comnam', 'shrcls',
       'tsymbol', 'naics', 'primexch', 'trdstat', 'secstat', 'permco',
       'compno', 'issuno', 'hexcd', 'hsiccd', 'cusip'],
      dtype='object')

In [ ]:
crsp = conn.raw_sql("""
                      select *
                      from crsp.dsf as a
                      left join crsp.dsenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/2021' and '12/31/2021'
                      and ABS(a.prc)*a.shrout>10000000
                      """, date_cols=['date'])

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 22))



StatementError: ignored

In [ ]:
crsp

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,naics,primexch,trdstat,secstat,permco,compno,issuno,hexcd,hsiccd,cusip
0,46603210,10026.0,7976.0,10433.0,3.0,2052.0,2021-12-30,157.654999,158.779999,158.279999,...,311930,Q,A,R,7976.0,60007929.0,10433.0,3.0,2052.0,46603210
1,46603210,10026.0,7976.0,10433.0,3.0,2052.0,2021-12-31,157.160004,159.231003,157.960007,...,311930,Q,A,R,7976.0,60007929.0,10433.0,3.0,2052.0,46603210
2,29402E10,10028.0,7978.0,22226.0,2.0,5094.0,2021-12-30,4.050000,4.169600,4.050000,...,423940,A,A,R,7978.0,60016522.0,22226.0,2.0,5094.0,29402E10
3,29402E10,10028.0,7978.0,22226.0,2.0,5094.0,2021-12-31,3.940000,4.110000,4.070000,...,423940,A,A,R,7978.0,60016522.0,22226.0,2.0,5094.0,29402E10
4,72913210,10032.0,7980.0,10437.0,3.0,3670.0,2021-12-30,95.839996,97.989998,95.839996,...,334418,Q,A,R,7980.0,60007933.0,10437.0,3.0,3670.0,72913210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14031,44409710,93419.0,53437.0,0.0,1.0,6798.0,2021-12-30,24.620001,25.129999,24.820000,...,531110,N,A,R,53437.0,0.0,0.0,1.0,6798.0,44409710
14032,44409710,93419.0,53437.0,0.0,1.0,6798.0,2021-12-31,24.639999,25.030001,24.709999,...,531110,N,A,R,53437.0,0.0,0.0,1.0,6798.0,44409710
14033,G3323L10,93427.0,53445.0,0.0,1.0,3827.0,2021-12-30,118.809998,122.419998,118.839996,...,333314,N,A,R,53445.0,0.0,0.0,1.0,3827.0,G3323L10
14034,G3323L10,93427.0,53445.0,0.0,1.0,3827.0,2021-12-31,118.260002,119.660004,118.470001,...,333314,N,A,R,53445.0,0.0,0.0,1.0,3827.0,G3323L10


In [ ]:
crsp['permno'].nunique()

permno    7021
permno    7021
dtype: int64

In [ ]:
crsp.to_csv('march_23_market_cap_10m.csv')

In [ ]:
l=abs(crsp['prc']*crsp['shrout'])

In [ ]:
crsp['MarketCap']=l

In [ ]:
crsp[crsp['siccd']>=100 and crsp['siccd']<=999]

ValueError: ignored